In [70]:
from importnb import Notebook, Partial, reload, load_ipython_extension, unload_ipython_extension
from nbformat import v4
from pathlib import Path
import shutil, os, functools
from pytest import fixture, mark


In [65]:
source ="""
foo = 42
assert {}
bar= 100
"""

In [66]:
def new_notebook(str='foo'):
    return v4.writes(v4.new_notebook(cells=[
            v4.new_code_cell(source.format(str))
        ]))

In [67]:
@fixture(scope='function')
def single_file(request):
    file = Path('foobar.ipynb')
    file.write_text(new_notebook())
    request.addfinalizer(functools.partial(os.remove, file))
    return file

In [71]:
@fixture
def clean_up_file(single_file, request):
    def clean_sys():
        import sys
        del sys.modules['foobar']
        sys.path_importer_cache.clear()
    request.addfinalizer(clean_sys)

In [72]:
def validate_reload(module):
    try:
        reload(module)
        assert False, """The reload should fail."""
    except:
        assert True, """Cannot reload a file outside of a context manager"""

    with Notebook():
        assert reload(module)

In [57]:
def test_single_file_with_context(clean_up_file):
    with Notebook():
        import foobar
    assert foobar.foo == 42 and foobar.bar == 100
    
    validate_reload(foobar)

In [ ]:
@fixture
def extension(clean_up_file, request):
    load_ipython_extension()
    request.addfinalizer(unload_ipython_extension)

In [ ]:
def test_single_with_extension(extension):
    import foobar
    assert foobar.foo == 42 and foobar.bar == 100

In [49]:
@mark.xfail
def test_single_file_relative(single_file):
    with Notebook():
        from . import foobar

In [46]:
@fixture
def single_directory(request):
    root = Path('a_test_package')
    root.mkdir(exist_ok=True)
    (root / 'foobar.ipynb').write_text(new_notebook())
    (root / 'failure.ipynb').write_text(new_notebook('False'))
    (root / 'py.py').write_text("""from . import foobar\nbaz = 'foobar'""")
    request.addfinalizer(functools.partial(shutil.rmtree, root))
    return root

In [ ]:
@mark.xfail
def test_single_file_without_context():
    import foobar

In [53]:
def test_package(single_directory):
    with Notebook():
        from a_test_package import foobar, py
        
    assert foobar.foo == 42 and foobar.bar == 100
    assert py.baz == 'foobar'
    assert py.foobar is foobar
    validate_reload(foobar)

In [53]:
@mark.xfail
def test_package_failure(single_directory):
    with Notebook():
        from a_test_package import failure

In [53]:
def test_package_failure_partial(single_directory):
    with Partial():
        from a_test_package import failure
        
    assert isinstance(failure.__exception__, AssertionError), """
    The wrong error was returned likely because of importnb."""

    from traceback import print_tb
    from io import StringIO
    s = StringIO()
    print_tb(failure.__exception__.__traceback__, file=s)
    assert """a_test_package/failure.ipynb", line 11, in <module>\n""" in s.getvalue(), """Traceback is not satisfied"""